# Separate the bilby samples by SNR thresholds
- SNR network 8, indiv minimum 2
- SNR network 9, indiv minimum 2
- SNR network 10, indiv minimum 2

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
cbcCatPath = "/global/homes/s/seanmacb/DESC/DESC-GW/gwStreetlights/data/mergers-w=Lum,n=1e6,FromSkySim50_withBilby.csv"
cbcCat = pd.read_csv(cbcCatPath)

In [2]:
# cbcCat

,galaxyID,redshiftHubble,ra,dec,mra,mdec,mass_ratio,chirp_mass,luminosity_distance,theta_jn,psi,phase,a_1,a_2,tilt_1,tilt_2,phi_12,phi_jl
0,10070200392465,2.787557,62.770779,-40.208431,172.823600,81.175717,0.784843,39.682598,23772.324284,1.470373,1.527574,6.111981,0.212367,0.924589,2.056307,1.538877,3.996148,5.284360
1,9814202166423,2.044943,62.394007,-37.236766,105.784619,41.289369,0.734108,65.747680,16252.753222,2.109578,2.587883,3.453603,0.569492,0.373157,1.799021,1.756383,1.602352,0.582240
2,10072100184865,1.834209,67.617224,-41.572680,241.762863,70.242273,0.496394,38.488675,14204.132784,0.446852,0.968015,1.227379,0.846142,0.460867,1.157485,1.211003,5.459455,1.385960
3,9816101164643,1.442409,66.938860,-36.626004,330.745078,-75.967953,0.830846,27.164701,10528.900213,1.544962,0.527480,0.915036,0.975962,0.591477,0.870990,1.903603,4.174135,5.098213
4,10070201792402,2.190634,61.436562,-41.042127,92.144000,-15.350534,0.588188,52.531755,17693.717694,0.870132,1.262305,0.295705,0.131235,0.189865,2.363448,1.695189,4.381864,4.531333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,10450101208010,1.435337,68.072960,-46.354699,309.861134,28.528358,0.904816,81.366434,10464.424991,1.661303,1.841819,4.483587,0.397964,0.825445,2.778606,0.812033,0.026667,2.791771
999996,9686101859930,1.115703,62.722171,-35.363638,199.277924,43.354578,0.806583,62.704688,7634.337758,1.212355,1.487383,0.540498,0.394093,0.380944,1.351863,1.442118,5.396874,1.606512
999997,10198100729664,1.624585,63.947168,-41.072617,254.927392,-53.865419,0.858758,37.984952,12213.869886,2.203670,1.740654,2.041428,0.214294,0.310846,2.760485,0.735742,1.010924,5.963968
999998,10071102123701,1.025592,64.389561,-40.703905,196.208747,85.264949,0.217823,33.333007,6870.398373,0.903281,1.388654,0.728124,0.542695,0.727033,1.712242,1.987625,1.386559,5.978017


In [3]:
import bilby as bb
import sys
sys.path.append("/global/homes/s/seanmacb/DESC/DESC-GW/gwStreetlights/utils")
import utils as ut
import numpy as np
bb.core.utils.log.setup_logger(outdir='.', label=None, log_level=30)

In [4]:
prior = bb.gw.prior.BBHPriorDict(aligned_spin=False) # The bbh prior, spins misaligned
cbcKeys = prior.sample().keys()

In [5]:
ifos = bb.gw.detector.InterferometerList(["H1","V1","L1"])
duration = 5
sampling_frequency = 2048
waveform_arguments = dict(waveform_approximant="IMRPhenomXP",  # waveform approximant name
                                  reference_frequency=50.0,  # gravitational waveform reference frequency (Hz)
                                 )

In [6]:
net_snr = []
indiv_snr = []

for ids,row in cbcCat.iterrows():
    injDict = {}
    for key in cbcKeys:
        if key in ("ra","dec"):
            injDict[key] = row["m"+key]
        else:
            injDict[key] = row[key] 
    injDict["geocent_time"] = ut.fakeGeoCentTime()
    ut.get_source_model("BBH")
    waveform_generator = bb.gw.waveform_generator.WaveformGenerator(sampling_frequency=sampling_frequency,duration=duration,
                                                                               frequency_domain_source_model=ut.get_source_model("BBH"),parameters=injDict,
                                                                               waveform_arguments=waveform_arguments,start_time=injDict["geocent_time"]-duration/2)
    
    ifos.set_strain_data_from_power_spectral_densities( # Set the strain
                duration=duration,
                sampling_frequency=sampling_frequency,
                start_time=injDict["geocent_time"] - duration/2,
            )
    
    # Compute SNR
    
    network_snr = 0
    indivSNR = []
    for ifo in ifos:
        signal = ifo.get_detector_response(
            waveform_polarizations=waveform_generator.frequency_domain_strain(injDict),
            parameters=injDict)
        single_snr_squared = ifo.optimal_snr_squared(signal).real
        indivSNR.append(np.sqrt(single_snr_squared))
        network_snr += single_snr_squared
    network_snr = np.sqrt(network_snr)

    net_snr.append(network_snr)
    indiv_snr.append(np.min(indivSNR))

/pscratch/sd/s/seanmacb/gwCosmoDesc/lib/python3.10/site-packages/lalsimulation/lalsimulation.py:8: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal


KeyboardInterrupt: 

In [ ]:
cbcCat["Individual SNR minimum"] = indiv_snr
cbcCat["Network SNR"] = net_snr

In [ ]:
cbcCat

In [ ]:
cbcCat.to_csv("/global/homes/s/seanmacb/DESC/DESC-GW/gwStreetlights/data/mergers-w=Lum,n=1e6,FromSkySim50_withBilby_andSNR.csv",index=False)